In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 6, Finished, Available)

In [2]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 7, Finished, Available)

In [3]:
submissions_path = "reddit-parquet/submissions/"
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 8, Finished, Available)

In [4]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(["Minecraft", "Eldenring","Genshin_Impact","pokemongo","RocketLeagueExchange","VALORANT","DestinyTheGame","CallOfDutyMobile","FIFA","pokemon"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 9, Finished, Available)

In [5]:
df_text = submissions_filtered.select("subreddit", "title", "selftext", "author","over_18","spoiler","is_video").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 10, Finished, Available)

In [6]:
from pyspark.sql.functions import concat_ws
df = df_text.withColumn("text", concat_ws(" ", df_text.title, df_text.selftext))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 11, Finished, Available)

In [7]:
df.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 12, Finished, Available)

+---------+--------------------+--------------------+---------------+-------+-------+--------+--------------------+
|subreddit|               title|            selftext|         author|over_18|spoiler|is_video|                text|
+---------+--------------------+--------------------+---------------+-------+-------+--------+--------------------+
|     FIFA|if you every get ...|                    |  Robert-Fulton|  false|  false|   false|if you every get ...|
|Minecraft|     Texting a house|So I’m building a...|     Dogeisagod|  false|  false|   false|Texting a house S...|
| VALORANT|       Killjoy Mains|Got bored and so ...|Status_Bet_8034|  false|  false|   false|Killjoy Mains Got...|
|Minecraft|Trident skeleton ...|                    |    Age_Natural|  false|  false|   false|Trident skeleton ...|
|  pokemon|the PokemonTogeth...|So several days a...|OneWhoGetsBread|  false|  false|   false|the PokemonTogeth...|
+---------+--------------------+--------------------+---------------+---

In [8]:
from pyspark.sql.functions import lower
df = df.withColumn("text_lower", lower(df.text))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 13, Finished, Available)

In [9]:
df = df.drop(*["title","selftext","text"])
df.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 14, Finished, Available)

+---------+---------------+-------+-------+--------+--------------------+
|subreddit|         author|over_18|spoiler|is_video|          text_lower|
+---------+---------------+-------+-------+--------+--------------------+
|     FIFA|  Robert-Fulton|  false|  false|   false|if you every get ...|
|Minecraft|     Dogeisagod|  false|  false|   false|texting a house s...|
| VALORANT|Status_Bet_8034|  false|  false|   false|killjoy mains got...|
|Minecraft|    Age_Natural|  false|  false|   false|trident skeleton ...|
|  pokemon|OneWhoGetsBread|  false|  false|   false|the pokemontogeth...|
+---------+---------------+-------+-------+--------+--------------------+
only showing top 5 rows



# assign keywords labels

In [10]:
keyword_lst = [
    ["bug","error","issue","glitch","crash","freeze","lag","problems","problem","corruption","restart"],
    ["strategies","strategy","tip","trick","tactic","review","playthrough","speedrun","trophy","trophies","speedrun","achievements","quests","mission","stuck","help"],
    ["story","storyline","plot","narrative","characters","dialogue","twist","protagonist","antagonist","villian","backstory","emotion","choice","suspense","sexy","waifu"],
    ["lore","world","mythology","culture","retcon","documents","race","racial","cultural","environment","environmental","artifacts","cosmology","asian","american","african","background"],
    ["trade","buy","sell","transaction","market","scam","valuation","negotiation","currency","credit","auction","rarity","tax","fee","traders","buyers","sellers","money","dollar"]
]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 15, Finished, Available)

In [11]:
from pyspark.sql.functions import when
df_label = df.withColumn("label",
                   when(col("text_lower").rlike("|".join(keyword_lst[0])), "bug report")
                   .when(col("text_lower").rlike("|".join(keyword_lst[1])), "gameplay discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[2])), "story related discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[3])), "lore discussion")
                   .when(col("text_lower").rlike("|".join(keyword_lst[4])), "game items trading")
                   .otherwise("other"))
df_label.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 16, Finished, Available)

+---------+---------------+-------+-------+--------+--------------------+-------------------+
|subreddit|         author|over_18|spoiler|is_video|          text_lower|              label|
+---------+---------------+-------+-------+--------+--------------------+-------------------+
|     FIFA|  Robert-Fulton|  false|  false|   false|if you every get ...|              other|
|Minecraft|     Dogeisagod|  false|  false|   false|texting a house s...|              other|
| VALORANT|Status_Bet_8034|  false|  false|   false|killjoy mains got...| game items trading|
|Minecraft|    Age_Natural|  false|  false|   false|trident skeleton ...|         bug report|
|  pokemon|OneWhoGetsBread|  false|  false|   false|the pokemontogeth...|gameplay discussion|
+---------+---------------+-------+-------+--------+--------------------+-------------------+
only showing top 5 rows



# check the distribution of each labels

In [18]:
df_label_cnt = df_label.groupBy("label").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 23, Submitted, Running)

In [21]:
df_label_cnt_tot = spark.createDataFrame(df_label_cnt).toPandas()
df_label_cnt_tot

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 26, Finished, Available)

,label,count
0,other,3925294
1,bug report,358867
2,game items trading,330996
3,gameplay discussion,240189
4,lore discussion,150275
5,story related discussion,86385


In [22]:
df_label_cnt_game = df_label.groupBy("subreddit","label").count().orderBy(col("count"), ascending=False).collect()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 27, Finished, Available)

In [23]:
df_label_cnt_sub = spark.createDataFrame(df_label_cnt_game).toPandas()
df_label_cnt_sub

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 28, Finished, Available)

,subreddit,label,count
0,Minecraft,other,671646
1,Eldenring,other,634060
2,Genshin_Impact,other,555686
3,pokemongo,other,373640
4,VALORANT,other,317794
5,CallOfDutyMobile,other,312899
6,FIFA,other,297790
7,pokemon,other,291050
8,DestinyTheGame,other,253196
9,RocketLeagueExchange,other,217533


In [24]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")
df_label_cnt_tot.to_csv(f"{CSV_DIR}/total_label_count.csv", index=False)
df_label_cnt_sub.to_csv(f"{CSV_DIR}/game_label_count.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 29, Finished, Available)

# train test validation split

In [18]:
df_ml = df_label.filter((col("label") == "bug report") | (col("label") == "gameplay discussion") | (col("label") == "story related discussion") | (col("label") == "lore discussion") | (col("label") == "game items trading"))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 23, Finished, Available)

In [32]:
df_ml = df_ml.withColumn("over_18_str", col("over_18").cast("string"))
df_ml = df_ml.withColumn("spoiler_str", col("spoiler").cast("string"))
df_ml = df_ml.withColumn("is_video_str", col("is_video").cast("string"))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 37, Finished, Available)

In [51]:
df_ml.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 56, Finished, Available)

+--------------------+---------------+-------+-------+--------+--------------------+-------------------+-----------+-----------+------------+
|           subreddit|         author|over_18|spoiler|is_video|          text_lower|              label|over_18_str|spoiler_str|is_video_str|
+--------------------+---------------+-------+-------+--------+--------------------+-------------------+-----------+-----------+------------+
|            VALORANT|Status_Bet_8034|  false|  false|   false|killjoy mains got...| game items trading|      false|      false|       false|
|           Minecraft|    Age_Natural|  false|  false|   false|trident skeleton ...|         bug report|      false|      false|       false|
|             pokemon|OneWhoGetsBread|  false|  false|   false|the pokemontogeth...|gameplay discussion|      false|      false|       false|
|RocketLeagueExchange|       Gloctane|   true|  false|   false|[xbox] [h] og tac...|gameplay discussion|       true|      false|       false|
|     

In [52]:
df_ml = df_ml.drop(*["over_18","spoiler","is_video"])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 57, Finished, Available)

In [53]:
df_ml2 = df_ml.sample(withReplacement=False, fraction=0.3, seed=42) # create the sample df

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 58, Finished, Available)

In [54]:
df_ml.printSchema

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 59, Finished, Available)

<bound method DataFrame.printSchema of DataFrame[subreddit: string, author: string, text_lower: string, label: string, over_18_str: string, spoiler_str: string, is_video_str: string]>

In [55]:
train_data, test_data, val_data = df_ml2.randomSplit([0.8, 0.18, 0.02], 24)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 60, Finished, Available)

In [27]:
print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of validation records : " + str(val_data.count()))
train_data.cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 56, 32, Finished, Available)

Number of training records: 932198
Number of testing records : 210806
Number of validation records : 23708


DataFrame[subreddit: string, title: string, selftext: string, text: string, text_lower: string, label: string]

# ML pipeline

In [14]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, MultilayerPerceptronClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 19, Finished, Available)

In [56]:
stringIndexer_label = StringIndexer(inputCol="label", outputCol="label_success")
stringIndexer_subreddit = StringIndexer(inputCol="subreddit", outputCol="subreddit_ix")
stringIndexer_author = StringIndexer(inputCol="author", outputCol="author_ix")
stringIndexer_adult = StringIndexer(inputCol="over_18_str", outputCol="over_18_ix")
stringIndexer_spoiler = StringIndexer(inputCol="spoiler_str", outputCol="spoiler_ix")
stringIndexer_video = StringIndexer(inputCol="is_video_str", outputCol="is_video_ix")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 61, Finished, Available)

In [57]:
onehot_subreddit = OneHotEncoder(inputCol="subreddit_ix", outputCol="subreddit_vec")
onehot_author = OneHotEncoder(inputCol="author_ix", outputCol="author_vec")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 62, Finished, Available)

In [58]:
vectorAssembler_features = VectorAssembler(
    inputCols=["subreddit_vec", "author_vec"], 
    outputCol= "str_feature")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 63, Finished, Available)

# Hyperparameter set-up

In [23]:
# random forest classifier hyperparameters
num_tree1 = 30
max_depth1 = 3
num_tree2 = 50
max_depth2 = 5

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 28, Finished, Available)

In [59]:
model_rf1 = RandomForestClassifier(labelCol="label_success", featuresCol="str_feature", numTrees=num_tree1, maxDepth=max_depth1)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 64, Finished, Available)

In [60]:
labelConverter = IndexToString(inputCol="prediction", 
                               outputCol="predictedLabel", 
                               labels= ["bug report", "gameplay discussion", "story related discussion","lore discussion","game items trading"])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 65, Finished, Available)

In [61]:
pipeline_model_rf = Pipeline(stages=[stringIndexer_label, 
                               stringIndexer_subreddit, 
                               stringIndexer_author, 
                               stringIndexer_adult,
                               stringIndexer_spoiler,
                               stringIndexer_video, 
                               onehot_author,
                               onehot_subreddit,
                               vectorAssembler_features, 
                               model_rf1, labelConverter])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 66, Finished, Available)

In [62]:
model_rf1 = pipeline_model_rf.fit(train_data)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 57, 67, Cancelled, Waiting)